In [4]:
PATH_TO_WORKING_DIR = 'f:/dss18/item_embedding_session_reco/'
PATH_TO_ORIGINAL_DATA = '././data/'
PATH_TO_PROCESSED_DATA = '././pre-train/'
file_2_tokenize_name = PATH_TO_PROCESSED_DATA + 'rcs15_sesss_tr.txt'
file_2_tokenize_name_test = PATH_TO_PROCESSED_DATA + 'rcs15_sesss_ts.txt'
tokenized_file_name = PATH_TO_PROCESSED_DATA + 'rcs15_sesss_tr_tk.txt'
tokenized_file_name_test = PATH_TO_PROCESSED_DATA + 'rcs15_sesss_ts_tk.txt'
tokenized_file_name_X = PATH_TO_PROCESSED_DATA + 'rcs15_sesss_tr_tkX.txt'
tokenized_file_name_Y = PATH_TO_PROCESSED_DATA + 'rcs15_sesss_tr_tkY.txt'
tokenized_file_name_test_X = PATH_TO_PROCESSED_DATA + 'rcs15_sesss_ts_tkX.txt'
tokenized_file_name_test_Y = PATH_TO_PROCESSED_DATA + 'rcs15_sesss_ts_tkY.txt'
glove_vectors_file_name = PATH_TO_PROCESSED_DATA + 'vectors_s_rcs15.txt'
glove_vocab_file_name = PATH_TO_PROCESSED_DATA + 'rcs15_s_vocab.txt'
vocab_size = 14386
sessions_train = 781486
sessions_test = 30726 
sequence_length = 3
sequence_len = 3
embedding_size = 50
do_val = 0.2
n_samples = 10
hidden_neurons = 30  #RNN layer output dim
num_iter = 2
N = 10  #recall@
maxW = N+3   #max item clicke we count

In [5]:
import pandas as pd
import numpy as np
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import Input, Layer, Dropout, Masking
from keras.layers import GRU, Activation
from keras.models import Model, Sequential
import numpy as np
import time
import argparse
#from keras.utils import to_categorical
import tensorflow as tf
import os

In [ ]:
import keras
keras.__version__

In [6]:
def to_categorical(y, nb_classes=None):
    '''Convert class vector (integers from 0 to nb_classes)
    to binary class matrix, for use with categorical_crossentropy.
    '''
    if not nb_classes:
        nb_classes = np.max(y)+1
    Y = np.zeros((len(y), nb_classes),dtype=np.bool)
    for i in range(len(y)):
        idx = y[i,0]
        Y[i, idx] = 1
    return Y

def generate(glove_vectors_file_name,glove_vocab_file_name):
    with open(glove_vocab_file_name, 'r') as f:
        words = [x.rstrip().split(' ')[0] for x in f.readlines()]
    with open(glove_vectors_file_name, 'r') as f:
        vectors = {}
        for line in f:
            vals = line.rstrip().split(' ')
            vectors[vals[0]] = [float(x) for x in vals[1:]]

    vocab_size = len(words)
    vocab = {w: idx for idx, w in enumerate(words)}
    ivocab = {idx: w for idx, w in enumerate(words)}

    vector_dim = len(vectors[ivocab[0]])
    W = np.zeros((vocab_size, vector_dim))
    for word, v in vectors.items():
        if word == '<unk>':
            continue
        W[vocab[word], :] = v

    # normalize each word vector to unit variance
    W_norm = np.zeros(W.shape)
    d = (np.sum(W ** 2, 1) ** (0.5))
    W_norm = (W.T / d).T
    return (W_norm, vocab, ivocab)

In [7]:
def run_experiment():

    W, vocab, ivocab = generate(glove_vectors_file_name, glove_vocab_file_name)
    
    print("loading train")
    y1 = np.loadtxt(tokenized_file_name_Y, dtype='int').reshape((sessions_train, 1))
    Y = to_categorical(y1, len(vocab) + 1)
    X = np.loadtxt(tokenized_file_name_X).reshape((sessions_train,sequence_len,embedding_size))
    out_n = len(vocab)+1

    #create the model here
    print("Creating model...")
    model = Sequential()
    print("Adding GRU ...")
    model.add(GRU(hidden_neurons, input_dim=embedding_size, return_sequences=False))
    print("Adding dropout ...")
    model.add(Dropout(do_val))
    print("adding output layer...")
    model.add(Dense(out_n, input_dim=hidden_neurons))
    print("adding activation...")
    model.add(Activation("softmax"))
    print("compiling...")
    model.compile(optimizer='adam',loss='categorical_crossentropy')
    print("compiled!")
    
    print(model.summary())

    data_size = X.shape[0]
    train_size = int(data_size)

    X_train = X[0:train_size, :]
    Y_train = Y[0:train_size]

    for II in range(1, num_iter):
        if II < 10:
            step = 1
        else:
            step = 10
        model.fit(X_train, Y_train, batch_size=100, nb_epoch=step, validation_split=0.05)

         # saving the model structures and parameters so it can be loaded later
        z1 = "m3_%d" % (II)
        json_string = model.to_json()
        open(z1+'_model_architecture.json', 'w').write(json_string)
        model.save_weights(z1+'_model_weights.h5')


In [9]:
os.chdir(PATH_TO_WORKING_DIR)
run_experiment()

loading train


MemoryError: 